<a href="https://colab.research.google.com/github/niamhgraham/Niamh/blob/master/ProspectInterview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prospect Football Interview Task

One of the main requirements of the Data Scientist role at Prospect is to use various datasets to create new and novel metrics to explain team and player performance. With that in mind and with the datasets provided, we’d like you to create a presentation explaining the different playing styles of the teams participating in the J League in 2024, using any modelling or statistical techniques of your choice.



We’re particularly interested to learn if you can tell us how they vary in their approaches in build-up and how they also look to defend. We’d also like you to do a deep dive on a single team of interest, analysing their strengths and weaknesses.



You’ll be required to present your presentation back to a panel representing the roles of a Sporting Director, Head of Recruitment and Head Coach. Please consider how your presentation should cater to these different stakeholders and what potential questions could come up when discussing how these teams play.

**Consideraions**

*   What metrics contribute to build up play
*   What metrics contribute to defensive play
*   Carry out a PCA on the metrics to try and find a defensive and build up
*   Look at how effective the strategies are by looking at how many chances/shots/xg team has and how many goals they have conceded.
*   




In [ ]:
import numpy as np
import json
import glob
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import os
import requests
import json
import math

In [ ]:
#load in data
match_data=json.load(sb_matches.json)
match_data=pd.json_normalize(match_data)

NameError: name 'sb_matches' is not defined

In [ ]:
# Load the JSON event data, inserting relevant file path
#with open('sb_events.json') as f:
#e_data = json.load(f)
# Load the JSON match data, inserting relevant file path
with open('matches.json') as f:
  m_data = json.load(f)
# Convert the JSON data to a DataFrame
#events_df = pd.json_normalize(e_data)
matches_df = pd.json_normalize(m_data)
#To make the naming convention correct
#events_df.columns = events_df.columns.str.replace(".name", "", regex=True)
#events_df.columns = events_df.columns.str.replace("[.]", "_", regex=True)
matches_df.columns = matches_df.columns.str.replace(".name", "", regex=True)
matches_df.columns = matches_df.columns.str.replace("[.]", "_", regex=True)

FileNotFoundError: [Errno 2] No such file or directory: 'matches.json'